# 如何使用FLGo并行运行联邦算法？ 

在前几节中，不同的运行器(i.e. runner)都是串行运行的，即只能一个runner结束之后，才能开启下一个的训练。且如果并行运行不同runner的话，不止要面临同时写好几个文件的问题，还需要手动观察GPU的状况，防止显存溢出导致程序终止，造成时间精力的浪费。

为了克服这些做实验的问题，这里把并行运行不同runner的功能封装到了`flgo.multi_init_and_run`函数中，由该函数可以同时运行多个runner，且自动管理GPU来保证成功运行所有的runner。该函数定义如下：

In [ ]:
def multi_init_and_run(runner_args:list, devices = [], scheduler=None):
    r"""
    Create multiple runners and run in parallel
    param:
        runner_args (list): each element in runner_args should be either a dict or a tuple or parameters
        devices (list): a list of gpu id
        scheduler (class flgo.experiment.device_scheduler.BasicScheduler): GPU scheduler
    return:
        a list of output results of runners
    Example:
        >>> from flgo.algorithm import fedavg, fedprox, scaffold
        >>> # create task 'mnist_iid' by flgo.gen_task if there exists no such task
        >>> task='./mnist_iid'
        >>> if os.path.exists(task): flgo.gen_task({'benchmark':{'name':'flgo.benchmark.mnist_classification'}, 'partitioner':{'name':'IIDPartitioner','para':{'num_clients':100}}}, task)
        >>> algos = [fedavg, fedprox, scaffold]
        >>> flgo.multi_init_and_run([{'task':task, 'algorithm':algo} for algo in algos], devices=[0])
    """
    ...

`flgo.multi_init_and_run`函数返回的结果为：所有传入的runner运行结束之后，由各自持有的logger所记录的输出字典。

`flgo.multi_init_and_run`：
* **runner_args**：列表类型，运行器配置（字典）的列表
* **devices（可选）**：列表类型，用于调度的gpu编号列表，例如\[0,1\]将指定0号卡和1号卡
* **scheduler（可选）**：类型为flgo.experiment.device_scheduler.BasicScheduler。用于指定GPU的调度器来自动调度任务。

`devices`当且仅当`scheduler`未指定时生效，此时`scheduler`将被自动指定为`flgo.experiment.device_scheduler.BasicScheduler`，不具备GPU自动管理的能力，所有runner会被一股脑放到各个gpu上，容易造成显存溢出。

为了让不同的runner可以安全地运行到结束，这里把`scheduler`留做一个可以自定义的接口，让用户可以自行定义如何调度GPU，并且提供了一个默认的gpu自动调度器的实现：`flgo.experiment.device_scheduler.AutoScheduler`。`AutoScheduler`调度器每隔一定周期检查gpu是否可用，若可用的话则把runner放到卡上去跑。其中这里判定gpu可用的详细定义为：若某张gpu的空闲显存大于所有已运行的runner的平均(或最大)的显存占用超过一定时间长度，则该gpu可用。

下面演示如何使用`flgo.multi_init_and_run`并行运行runner

## Example 1. 并行运行不同参数

In [ ]:
import flgo
import flgo.algorithm.fedavg as fedavg
import flgo.experiment.device_scheduler as ds

In [ ]:
# 创建联邦任务
task = './test_cifar10'
# 为每个runner参数指定：任务、算法、参数，该字典的内容应与flgo.init的参数一致，未填写的项将使用默认值。
runner_dict = [{'task':task, 'algorithm':fedavg, 'option':{'lr':0.01*i, 'no_log_console':True, 'num_rounds':5}} for i in range(1,6)]
# 指定调度器为flgo.experiment.device_scheduler.AutoScheduler，并指定可用的卡为0号卡
asc = ds.AutoScheduler([0])
# 并行运行命令，可以看到子进程报错退出后，并不会导致主进程也退出，相应的runner将会在合适的时刻被再次运行，直至所有runner成功运行结束。
flgo.multi_init_and_run(runner_dict, scheduler=asc)

## Example 2. 并行运行不同算法

In [ ]:
# 创建联邦任务
from flgo.algorithm import fedavg, fedprox
task = './test_cifar10'
algos = [fedavg, fedprox]
runner_dict = [{'task':task, 'algorithm':a, 'option':{'lr':0.01, 'no_log_console':True, 'num_steps':1,'num_rounds':2}} for a in algos]
# 指定调度器为flgo.experiment.device_scheduler.AutoScheduler，并指定可用的卡为0号卡
asc = ds.AutoScheduler([0])
res = flgo.multi_init_and_run(runner_dict, scheduler=asc)

In [ ]:
import matplotlib.pyplot as plt
for r in res:
    plt.plot(list(range(len(r['test_loss']))), r['test_loss'], label = r['option']['algorithm'])
plt.legend()
plt.show()

# 如何使用FLGo自动网格搜索调参？ 

网格搜索通过`flgo.tune`来实现，它跟上述`flgo.multi_init_and_run`的并行机制是一致的，通过指定`scheduler`来自动并行运行命令。两者的区别在于传入的参数不同，其中`flgo.tune`的输入`flgo.init`具有相同的形式如下，和`flgo.init`的参数区别在于`option`

In [ ]:
# option定义示例如下，其中每个关键字被指定为列表，而不是数值（单个数值将被视为长度为1的列表）。调参的范围为所有列表的值排列组合构成的网格
option = {
    'learning_rate':[0.1, 0.01, 0.05],
    'batch_size':[16,32],
    'num_epochs':[1,5],
}
# tune函数声明：“
def tune(task: str, algorithm, option: dict = {}, model=None, Logger: flgo.experiment.logger.BasicLogger = flgo.experiment.logger.tune_logger.TuneLogger, Simulator: BasicSimulator=flgo.simulator.DefaultSimulator, scene='horizontal', scheduler=None):
    """
        Tune hyper-parameters for one task and one algorithm in parallel.
        :param
            task (str): the dictionary of the federated task
            algorithm (module || class): the algorithm will be used to optimize the model in federated manner, which must contain pre-defined attributions (e.g. algorithm.Server and algorithm.Client for horizontal federated learning)
            option (dict): the dict whose values should be of type list to construct the combinations
            model (module || class): the model module that contains two methods: model.init_local_module(object) and model.init_global_module(object)
            Logger (class): the class of the logger inherited from flgo.experiment.logger.BasicLogger
            Simulator (class): the class of the simulator inherited from flgo.simulator.BasicSimulator
            scene (str): 'horizontal' or 'vertical' in current version of FLGo
            scheduler (instance of flgo.experiment.device_scheduler.BasicScheduler): GPU scheduler that schedules GPU by checking their availability
        """
    ...

此外，传入`flgo.tune`函数的logger必须在`logger.log_once`方法中，记录关键字`val_loss`到`logger.output`中，调参在验证集上进行。因此默认调用的logger为`flgo.experiment.logger.tune_logger.TuneLogger`。下面演示如何使用`flgo.tune`来自动调参。


## Example 3. 自动调参 

In [19]:
option = {
    'learning_rate':[0.1, 0.01, 0.05],
    'batch_size':[16],
    'num_steps':[1,5],
    'num_rounds': 5,
    'no_log_console': True
}
flgo.tune(task, fedavg, option, scheduler = asc)
# 在该简单网格上，最后调参输出结果如下

Process 14314 was created for args ('./test_cifar10', 'flgo.algorithm.fedavg', {'learning_rate': 0.1, 'batch_size': 16, 'num_steps': 1, 'num_rounds': 5, 'no_log_console': True, 'log_file': True, 'gpu': 0}, None, <class 'flgo.experiment.logger.tune_logger.TuneLogger'>, <class 'flgo.simulator.default_simulator.Simulator'>, 'horizontal')
Process 14523 was created for args ('./test_cifar10', 'flgo.algorithm.fedavg', {'learning_rate': 0.1, 'batch_size': 16, 'num_steps': 5, 'num_rounds': 5, 'no_log_console': True, 'log_file': True, 'gpu': 0}, None, <class 'flgo.experiment.logger.tune_logger.TuneLogger'>, <class 'flgo.simulator.default_simulator.Simulator'>, 'horizontal')
Process 14621 was created for args ('./test_cifar10', 'flgo.algorithm.fedavg', {'learning_rate': 0.01, 'batch_size': 16, 'num_steps': 1, 'num_rounds': 5, 'no_log_console': True, 'log_file': True, 'gpu': 0}, None, <class 'flgo.experiment.logger.tune_logger.TuneLogger'>, <class 'flgo.simulator.default_simulator.Simulator'>, 'h

可以看到最优参数为步长为0.05，批大小为16，本地更新步数为5，在第5个通信轮次达到最低验证集损失。

## 如何使用早停机制？ 

相较于传统机器学习中，判定超过多少个epoch验证集损失未改善便停止训练，在联邦中这里使用的是超过多少个通信轮次验证集损失未改善便停止训练。为达到此目的，仅需要在option中额外指定早停的连续未改进轮数如下：

In [ ]:
option = {
    'learning_rate':[0.1, 0.01],
    'batch_size':[16],
    'num_steps':[1],
    'num_rounds': 1000,
    'no_log_console': True，
    'early_stop': 10
}
flgo.tune(task, fedavg, option, scheduler = asc)

在实践中，联邦由于noniid等因素的影响，训练波动很大，连续数十个通信轮数验证集损失没改进也不代表过拟合了。因此early_stop可以设的很大很大，例如500。